In [1]:
import dask.dataframe as dd
import numpy as np
from distributed import Client
from pathlib import Path


def _save_datasets(train, test, outdir: Path):
    """Save data sets into nice directory structure and write SUCCESS flag."""
    out_train = outdir / 'train.parquet/'
    out_test = outdir / 'test.parquet/'
    flag = outdir / '.SUCCESS'

    train.to_parquet(str(out_train))
    test.to_parquet(str(out_test))

    flag.touch()

in_csv = "../data_root/raw/wine_dataset.csv"
out_dir = "../data_root/processed/"
out_dir = Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

# Connect to the dask cluster
#c = Client('dask-scheduler:8786')

# load data as a dask Dataframe if you have trouble with dask
# please fall back to pandas or numpy
ddf = dd.read_csv(in_csv, blocksize=1e6)

# we set the index so we can properly execute loc below
ddf = ddf.set_index('Unnamed: 0')

# trigger computation
n_samples = len(ddf)

# TODO: implement proper dataset creation here
# http://docs.dask.org/en/latest/dataframe-api.html

# split dataset into train test feel free to adjust test percentage
idx = np.arange(n_samples)
test_idx = idx[:n_samples // 10]
test = ddf.loc[test_idx]

train_idx = idx[n_samples // 10:]
train = ddf.loc[train_idx]

_save_datasets(train, test, out_dir)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Plot / Graph stuffs
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import sklearn.metrics as metrics #import confusion_matrix, classification_report

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../data_root/raw"]).decode("utf8"))

wine_dataset.csv



In [22]:
selected_data = ddf[['country', 'description', 'points', 'price', 
    'province', 'title', 'variety','winery']]

data = selected_data.dropna()
print("Number of rows after removing missing data:" , len(data))

# trigger computation
n_samples = len(data)

# TODO: implement proper dataset creation here
# http://docs.dask.org/en/latest/dataframe-api.html

# split dataset into train test feel free to adjust test percentage


Number of rows after removing missing data: 9318


In [5]:
cat_features = [
    'country',
    'province',
    'variety',
    'winery',
]

num_features = [
    'price',
    'description_length'
]

labels = ['points']

In [23]:

data = data.assign(description_length = data['description'].apply(len))

X = data[num_features]
y = data[labels]

idx = np.arange(n_samples)

test_idx = idx[:n_samples // 10]
X_test = X.loc[test_idx]
y_test = y.loc[test_idx]

train_idx = idx[n_samples // 10:]
X_train = X.loc[train_idx]
y_train = y.loc[train_idx]

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

# Testing model
predictions = rfr.predict(X_test)

/home/philip/.virtualenvs/ml/lib/python3.6/site-packages/dask/dataframe/core.py:2955: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('description', 'int64'))

  warnings.warn(meta_warning(meta))
/home/philip/.virtualenvs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/philip/.virtualenvs/ml/lib/python3.6/site-packages/dask/dataframe/methods.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an al

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [21]:
data.isna().sum()

country               0
description           0
points                0
price                 0
province              0
title                 0
variety               0
winery                0
description_length    0
dtype: int64